In [ ]:
#default_exp sentryUtil

# sentryUtil

In [ ]:
#hide
import sentry_sdk
sentry_sdk.init(
    "https://0bab4064545e4a95b35d4e73f20f7632@o839457.ingest.sentry.io/5894971",

    # Set traces_sample_rate to 1.0 to capture 100%
    # of transactions for performance monitoring.
    # We recommend adjusting this value in production.
    traces_sample_rate=1.0
)

In [ ]:
#export
from sentry_sdk import add_breadcrumb, capture_exception, capture_message
from nicHelper.exception import traceback
from typing import Any
from copy import deepcopy
from nicHelper.exception import errorString
import functools, sentry_sdk, inspect

In [ ]:
#export
def logSentry(message:str, data:Any = (lambda :{})(), level:str = 'info', section:str='main'):
  '''
    just add docs for ease of logging to sentry
    Input:
      message ::str:: required :: message to send to sentry
      data ::dict:: optional :: and object to send to sentry (default is an empty dict)
      level ::str::optional:: log level (default:info)
      section ::str::optional:: section name or function name (default: main)
    Response:
      Bool:: true means logged properly, false for error, print error message to console
  '''
  try:
    add_breadcrumb(
      category=section,
      data={'data':deepcopy(data)},
      level=level,
      message=message
    )
    return True
  except Exception as e:
    print(message, data, level, section)
    print(f'error is {e}, {traceback()}')
    return False

In [ ]:
logSentry('hello', {'hello':'this is a test'}, level='error', section = 'test')
capture_message('testing')

'35bc2f85abe84f889768ba445ffb4c7e'

## full example

In [ ]:
from nicHelper.sentryUtil import logSentry
logSentry('this is a test', {'testobject': 'testvalue'})

True

# autologging to sentry
require sentry_sdk

In [ ]:
#export
def autologSentry(originalFunction=None, verbose=False):
  def outer(f):
    @functools.wraps(f)
    def inner(*args,**kwargs):
      try:
        r = f(*args,**kwargs)
        message = str(f.__name__)+str(inspect.signature(f))
        sentry_sdk.add_breadcrumb(category=f.__name__, message=message, data={
          'args': args, 'kwargs': kwargs, 'output':r }, level='info')
        if verbose: print(f.__name__, message, { 'args': args, 'kwargs': kwargs, 'output':r })
        return r
      except Exception as e:
        sentry_sdk.add_breadcrumb(category=f.__name__, message=errorString(), data={
          'args': args, 'kwargs': kwargs, 'error': str(e) }, level='critical')
        if verbose: print(f.__name__, errorString(), {'args': args, 'kwargs': kwargs, 'error':str(e)} )
        raise e
    return inner
  if originalFunction: return outer(originalFunction)
  return outer

In [ ]:
@autologSentry(verbose=True)
def testFunction(input_):
  '''this is crazy'''
  return input_ + '1'

testFunction('123')

testFunction testFunction(input_) {'args': ('123',), 'kwargs': {}, 'output': '1231'}


'1231'

In [ ]:
help(testFunction)

Help on function testFunction in module __main__:

testFunction(input_)
    this is crazy



In [ ]:
from dataclasses import dataclass
@dataclass
class TestClass:
    @autologSentry
    def test(self, input_):
        return input_

In [ ]:
TestClass().test('test')

'test'

In [ ]:
help(TestClass.test)

Help on function test in module __main__:

test(self, input_)

